In [59]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer, LabelEncoder
from sklearn import metrics
import os

In [60]:
#### load data files ###

# Change file name prefix here and you are good to go:D
# prefix = "train_"
# prefix = "A_"

# Generate data file name list
file_names = ["Application.csv","Personas.csv","History_Application.csv","History_Payment.csv"]
# for idx, val in enumerate(file_names):
#     file_names[idx] = prefix + val  

# load data files
prefix = "train_"
d01 = pd.read_csv(f'./data/{prefix}{file_names[0]}')
d02 = pd.read_csv(f'./data/{prefix}{file_names[1]}')
d03 = pd.read_csv(f'./data/{prefix}{file_names[2]}')
d04 = pd.read_csv(f'./data/{prefix}{file_names[3]}')
prefix = "A_"
# load data files
d05 = pd.read_csv(f'./data/{prefix}{file_names[0]}')
d06 = pd.read_csv(f'./data/{prefix}{file_names[1]}')
d07 = pd.read_csv(f'./data/{prefix}{file_names[2]}')
d08 = pd.read_csv(f'./data/{prefix}{file_names[3]}')

# Concat
d1 = pd.concat([d01, d05], axis=0)
d2 = pd.concat([d02, d06], axis=0)
d3 = pd.concat([d03, d07], axis=0)
d4 = pd.concat([d04, d08], axis=0)

In [61]:
### Functions

# 取一个列的众数
def get_mode(df, col):
    return pd.Series(df[col]).mode()[0]

# 置多个列的空值为所属列0
def fill_zero(df, cols):
    dict = {}
    for col in cols:
        dict[col] = 0
    return df.fillna(dict)

# 置多个列的空值为所属列的平均数
def fill_mean(df, cols):
    dict = {}
    for col in cols:
        dict[col] = df[col].mean()
    return df.fillna(dict)

# 置多个列的空值为所属列的众数
def fill_mode(df, cols):
    dict = {}
    for col in cols:
        dict[col] = get_mode(df, col)
    return df.fillna(dict)

# 把多个离散型的列转化为one-hot编码列
def dummy_catgorical(df, cols, keep_n_cats):
    print('initial num of cols: ' + str(len(df.columns.to_list())))
    for col in cols:
        # 跳过离散变量太多的
        idx_list = df[col].value_counts().index.to_list()
        print(col+'has'+str(len(idx_list)))
        if (len(idx_list) > keep_n_cats):
            print(' col含有过多值, skipping')
        else:
            # 生成one-hot编码列
            df = pd.get_dummies(df, columns=[col])
    return df

# 打印一个df的na值
def print_na(df):
    # 检查d12 nan值
    df_na = df.isna().sum()
    for idx in df_na.index.to_list():
        if (df_na[idx]!=0):
            print(str(idx)+' '+str(df_na[idx]))


In [62]:
### intersection
# ''' d01&d02 '''
# id_train = set(d01['申请编号'])
# len(id_train)
# id_test = set(d02['申请编号'])
# len(id_test)
# inter = id_train.intersection(id_test)
# len(inter)

In [63]:
### Merge d1 d2 ###
uid = '申请编号'
d12 = pd.merge(d1, d2, how='inner', on=uid)
d12.shape

(161511, 62)

In [64]:
### d12 异常值处理
d12.loc[(d12['孩子个数'] == -1), '孩子个数'] = get_mode(d12, '孩子个数')

#TODO 距申请日期>0, 置平均值
d12_cols = {}
d12_cols['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']

for col in d12_cols['_dt']:
    mean = d12[col].mean()
    d12.loc[d12[col] > 0, col] = mean

### Outlier：Get ###
low = .01
high = .99
quant_df = d12.drop([uid], axis=1).quantile([low, high]).T
quant_df.to_csv('./tmp/1_d12_outlier.csv')

### Outlier：Apply ###

outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)
outliers.shape
outliers.head()

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

# def set_outlier(col):
#     if col < col_min or col > col_max:
#         col = np.NaN
#     return col
    
for col in outliers.index:
# for col in '出生日期距申请日期天数', '社交圈违约信息1_2':
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    d12[col] = d12[col].apply(set_outlier)
    
# d12.to_csv(f'./tmp/2_eda_outlier.csv')

(61, 2)

,min,max
地址是否一致标志1,0,1
地址是否一致标志2,0,1
地址是否一致标志3,0,1
地址是否一致标志4,0,1
地址是否一致标志5,0,1


In [65]:
# 检查d12 nan值

d12.shape

'''d12 nan值'''
print_na(d12)

# d12['收入类型'].value_counts(dropna=False)
# d12_dum.columns.to_list()

(161511, 62)

'd12 nan值'

贷款年金 8
商品价格 140
陪同申请人 675
外部评分 90682
贷款申请前1小时内征信查询次数 22217
贷款申请前1天内征信查询次数 22217
贷款申请前1周内征信查询次数 22217
贷款申请前1个月内征信查询次数 22217
贷款申请前1个季度内征信查询次数 22217
贷款申请前1年内征信查询次数 22217
有车时间 106448
职业 48412
家庭成员数 1
居住信息1 107830
居住信息2 113207
居住信息3 81643
居住信息4 78587
社交圈违约信息2_2 516
社交圈违约信息2_1 516
社交圈违约信息1_2 516
社交圈违约信息1_1 516


In [66]:
### d12 数据空值处理 ###

# 插补平均值
nan_to_mean = [
    '贷款年金',
    '商品价格',
    '外部评分',
    '居住信息1',
    '居住信息2',
    '居住信息3',
    '居住信息4',
    '有车时间'
]
#TODO: 空值插补mean
d12 = fill_mean(d12, nan_to_mean)

nan_to_zero = [
    '贷款申请前1小时内征信查询次数',
    '贷款申请前1天内征信查询次数',
    '贷款申请前1周内征信查询次数',
    '贷款申请前1个月内征信查询次数',
    '贷款申请前1个季度内征信查询次数',
    '贷款申请前1年内征信查询次数'
]
#TODO: 空值插补0
d12 = fill_zero(d12, nan_to_zero)

#TODO: 空值插补mean
d12 = fill_mean(d12, nan_to_mean)

nan_to_mode = [
    '家庭成员数',
    '社交圈违约信息2_2',
    '社交圈违约信息2_1',
    '社交圈违约信息1_2',
    '社交圈违约信息1_1',
    '陪同申请人',
    '职业'
]
#TODO: 空值插补mode
d12 = fill_mode(d12, nan_to_mode)

# 检查d12 nan值
'''d12 nan值'''
print_na(d12)

'd12 nan值'

In [67]:
### d12 类型变量转dummy列 ###
cat_to_dummy = [
    '陪同申请人',
    '收入类型',
    '教育程度',
    '居住状态',
    '职业',
    '单位类型'
]

d12_dum = d12.copy()
d12_dum = dummy_catgorical(d12_dum, cat_to_dummy, 20)

d12.shape
d12_dum.shape
'''d12_dum nan值'''
print_na(d12_dum)

initial num of cols: 62
陪同申请人has7
收入类型has8
教育程度has5
居住状态has6
职业has18
单位类型has58
 col含有过多值, skipping


(161511, 62)

(161511, 101)

'd12_dum nan值'

In [68]:
### d12_dum 处理col含有过多值
if ('单位类型' in d12_dum.columns):
    d12_dum.drop(columns='单位类型',inplace=True)

In [69]:
### d3过滤

# 删除特征'是否为最后一次申请记录'
if ('是否为最后一次申请记录' in d3.columns):
    d3.drop(columns='是否为最后一次申请记录',inplace=True)

In [70]:
# 检查d3 nan值
# d3.shape
# d3['首付时间'].value_counts()
# '''d3 nan值'''
# print_na(d3)

In [71]:
### d3数据清洗 - 金额型 ###

# 异常值置NaN
d3.loc[(d3['首付时间'] == 292204.0), '首付时间'] = np.NaN
d3.loc[(d3['首付金额'] == 11), '首付金额'] = 0
d3.loc[(d3['贷款年金'] == 11), '贷款年金'] = 0
d3.loc[(d3['商品价格'] == 11), '商品价格'] = 0
d3.loc[(d3['申请额度'] == 11), '申请额度'] = 0
d3.loc[(d3['信用额度'] == 11), '信用额度'] = 0

# d3['销售区域_count'] = d3.groupby('销售区域')['销售区域'].transform('count')
# d3.loc[d3['销售区域_count'] < 5000, '销售区域'] = np.NaN

# 插补平均值
nan_to_zero = [
    '贷款年金',
    '首付金额',
    '商品价格',
    '首付时间',
    '首付率',
    '利率1',
    '利率1',
    '利率2'
]
#TODO: 空值插补0
d3 = fill_zero(d3, nan_to_zero)

### d3数据清洗 - 布尔型 ###
# d3_cols['bool'] = [
#     '是否为当天最后一次申请记录']

### d3数据清洗 - 计数型 ###
# d3_cols['cnt'] = [
#     '贷款期数']

d3.shape
# '''d3 nan值'''
# print_na(d3)

(744984, 31)

In [72]:
### d3 自定义特征

# '拟实际应还贷款金额'
d3.eval('拟实际应还贷款金额 = 贷款年金 * 贷款期数', inplace=True)
d3['拟实际应还贷款金额'].where(d3['贷款期数']>0, other=d3['贷款年金'], inplace=True)

# '信用额度 / 申请额度'
d3.eval('信用额度比 = 信用额度 / 申请额度', inplace=True)

d3.shape
d3.columns.to_list()

(744984, 33)

['历史申请编号',
 '申请编号',
 '贷款类型',
 '贷款年金',
 '申请额度',
 '信用额度',
 '首付金额',
 '商品价格',
 '申请周内日',
 '申请时点',
 '是否为当天最后一次申请记录',
 '首付率',
 '利率1',
 '利率2',
 '贷款用途',
 '合同状态',
 '历史贷款授信距本次申请时间',
 '付款方式',
 '拒绝原因',
 '陪同申请人',
 '客户类型',
 '商品类别',
 '组合类型',
 '产品类型',
 '获客渠道',
 '销售区域',
 '行业',
 '贷款期数',
 '收益级别',
 '产品组合',
 '首付时间',
 '拟实际应还贷款金额',
 '信用额度比']

In [73]:
### d3 类型变量转dummy列 ###

cat_to_dummy = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
    '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
    '销售区域',
    '行业',
    '收益级别',
    '产品组合',
    '申请时点',
    '申请周内日'
]

d3 = dummy_catgorical(d3, cat_to_dummy, 27)

# 过滤合同状态为'非拒绝'的样本
d3a = d3[d3['合同状态_2']==0]

# 取合同状态为'拒绝'的样本
d3d = d3[d3['合同状态_2']==1]

'''d3a'''
d3a.shape
'''d3d'''
d3d.shape

initial num of cols: 33
贷款类型has3
贷款用途has25
合同状态has4
付款方式has4
拒绝原因has9
陪同申请人has7
客户类型has4
商品类别has27
组合类型has5
产品类型has3
获客渠道has8
销售区域has1665
 col含有过多值, skipping
行业has11
收益级别has5
产品组合has17
申请时点has12
申请周内日has7


'd3a'

(612366, 168)

'd3d'

(132618, 168)

In [74]:
### d3 处理col含有过多值

if ('销售区域' in d3.columns):
    d3.drop(columns='销售区域',inplace=True)
    d3a.drop(columns='销售区域',inplace=True)
    d3d.drop(columns='销售区域',inplace=True)

In [75]:
### d3 group by ###

d3_cols = {}
# 金额型
d3_cols['curr'] = [
    '贷款年金',
    '申请额度',
    '信用额度',
    '首付金额',
    '商品价格']

# 类别型
d3_cols['cat'] = [
    '贷款类型',
    '贷款用途',
    '合同状态',
    '付款方式',
#     '拒绝原因',
    '陪同申请人',
    '客户类型',
    '商品类别',
    '组合类型',
    '产品类型',
    '获客渠道',
#     '销售区域',
    '行业',
    '收益级别',
    '产品组合']

# 时点型
d3_cols['time_point'] = [
    '申请周内日',
    '申请时点']

# 时长型
d3_cols['time_span'] = [
    '历史贷款授信距本次申请时间',
    '首付时间']

# 布尔型
d3_cols['bool'] = [
#     '是否为最后一次申请记录',
    '是否为当天最后一次申请记录']

# 概率型
d3_cols['prob'] = [
    '首付率',
    '利率1',
    '利率2']

# 计数型
d3_cols['cnt'] = [
    '贷款期数']

# d3a
# d3a_agg_dict = {'历史申请编号':'count'}

# 数值型 - 金额
# curr_agg = ['mean', 'max', 'min']
# for col in d3_cols['curr']:
#     d3a_agg_dict[col] = curr_agg
d3a_agg_dict = {
    '历史申请编号':['count'],
    '贷款年金':['mean', 'max', 'min', 'sum'],
    '申请额度':['mean', 'max', 'min', 'sum'],
    '信用额度':['mean', 'max', 'min', 'sum'],
    '首付金额':['mean', 'max', 'min'],
    '商品价格':['mean', 'max', 'min'],
    '拟实际应还贷款金额':['mean','max', 'min', 'sum'],
    '信用额度比':['mean', 'max', 'min']
}
    
# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d3_cols['time_span']:
    d3a_agg_dict[col] = time_span_agg

# 数值型 - 概率
prob_agg = ['mean', 'max', 'min']
for col in d3_cols['prob']:
    d3a_agg_dict[col] = prob_agg

# 离散型 - 类型
cat_agg = ['sum', 'mean']
for col1 in d3a.columns:
    for col2 in d3_cols['cat']:
        if (col2 in col1):
            d3a_agg_dict[col1] = cat_agg

# 离散型 - 时点
time_point_agg = ['sum', 'mean']
for col1 in d3a.columns:
    for col2 in d3_cols['time_point']:
        if (col2 in col1):
            d3a_agg_dict[col1] = time_point_agg


# groupby合同状态为非拒绝的
d3a = d3a.groupby('申请编号').agg(d3a_agg_dict)
# 还原列名
d3a.columns = pd.Series(d3a.columns.tolist()).apply(pd.Series).sum(axis=1)

# d3d
d3d_agg_dict = {'历史申请编号':['count']}

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d3_cols['curr']:
    d3d_agg_dict[col] = curr_agg
    
# 离散型 - 类型
cat_agg = ['sum', 'mean']
for col in d3d.columns:
    if ('拒绝原因' in col):
        d3d_agg_dict[col] = cat_agg

# groupby合同状态为拒绝的
d3d = d3d.groupby('申请编号').agg(d3d_agg_dict)
# 还原列名
d3d.columns = pd.Series(d3d.columns.tolist()).apply(pd.Series).sum(axis=1)

# d3all
d3all_agg_dict = {
    '历史申请编号':['count'],
    '贷款年金':['sum'],
    '申请额度':['sum'],
    '信用额度':['sum']
}

# groupby全部
d3all = d3.groupby('申请编号').agg(d3all_agg_dict)
# 还原列名
d3all.columns = pd.Series(d3all.columns.tolist()).apply(pd.Series).sum(axis=1)

d3a.shape
# d3a.columns.to_list()
d3d.shape
# d3d.columns.to_list()
d3all.shape
# d3all.columns.to_list()

(152768, 325)

(53598, 34)

(153006, 4)

In [76]:
### d3a merge d3all
d3a_all = pd.merge(d3a, d3all, how='inner', on='申请编号')
d3a_all.shape

# d3a_all自定义特征
d3a_all.eval('拟申请核准率 = 历史申请编号count_x / 历史申请编号count_y', inplace=True)
d3a_all.eval('贷款年金sum比 = 贷款年金sum_x / 贷款年金sum_y', inplace=True)
d3a_all.eval('申请额度sum比 = 申请额度sum_x / 申请额度sum_y', inplace=True)
d3a_all.eval('信用额度sum比 = 信用额度sum_x / 信用额度sum_y', inplace=True)
d3a_all.eval('信用比申请额度 = 信用额度sum_y / 申请额度sum_y', inplace=True)

drop_list = ['贷款年金sum_x', '贷款年金sum_y', '申请额度sum_x', '申请额度sum_y', '信用额度sum_x', '信用额度sum_y']
d3a_all.drop(columns=drop_list,inplace=True)

d3a_all.shape
# d3a_all.columns.to_list()

(152768, 329)

(152768, 328)

In [77]:
### merge d12 and d3a_all
d12.shape
d3a_all.shape
d12_d3a_all = pd.merge(d12, d3a_all, how='left', on='申请编号')
d12_d3a_all.shape
# print_na(d12_d3a_all)

#TODO: 空值插补0
d12_d3a_all = d12_d3a_all.fillna(0)

(161511, 62)

(152768, 328)

(161511, 390)

In [88]:
### d12_d3a_all 自定义特征

# '是否有非拒绝历史申请记录'
d12_d3a_all['有非拒历史申请'] = d12_d3a_all['历史申请编号count_x'].apply(lambda x: 1 if x > 0 else 0 )

In [20]:
# # 检查d4 nan值
# '''d4 nan值'''
# print_na(d4)

# # d4.head()
# d0 = pd.read_csv('./data/train_label.csv')
# # d0.shape
# # d0.head()

# d40 = pd.merge(d4, d0, how='left', on='申请编号')

# d40['分期付款日期'].value_counts()
# pd.DataFrame(d40.loc[d40['标签']==0,'分期付款日期'].value_counts().sort_index())

# pd.DataFrame(d40.loc[d40['分期付款日期']==np.NaN,'标签'].value_counts().sort_index())

# d4.groupby(['申请编号','历史申请编号','有支付行为的分期付款']).agg({'本期贷款金额':'sum'}).reset_index()


In [89]:
### d4 缺失值插补

# for col in d4.columns:
#     d4[col].value_counts()

# 距申请日期>0, 置平均值
# d4_cols = {}
# d4_cols['_dt'] = [
#     '分期付款实际支付时间']
    
nan_to_mean = [
    '分期付款实际支付时间',
    '本期还款金额'
]

#TODO 空值插补0
d4 = d4.fillna(0)

In [90]:
### d4 自定义特征

d4.eval('分期付款支付时间差 = 分期付款实际支付时间 - 分期付款应支付时间', inplace=True)
# d4['分期付款应支付时间'].value_counts()
# d4['分期付款实际支付时间'].value_counts()

In [91]:
### d4 group by phase 1 ###

d41_agg_dict = {
    '本期贷款金额':['sum'],
    '本期还款金额':['mean'],
    '分期付款支付时间差':['mean']
}
d41 = d4.groupby(['申请编号','历史申请编号','有支付行为的分期付款']).agg(d41_agg_dict)
d41.columns = pd.Series(d41.columns.tolist()).apply(pd.Series).sum(axis=1)
d41.shape
d41.head()


(5719219, 3)

本期贷款金额sum  本期还款金额mean  分期付款支付时间差mean
申请编号 历史申请编号 有支付行为的分期付款                                      
0    779169 2              1100.0      1100.0          -14.0
            3              1100.0      1100.0          -13.0
            4              1100.0      1100.0            1.0
            5              1100.0      1100.0          -23.0
            7              1100.0      1100.0           -1.0

In [92]:
### d4 自定义特征
# d41['分期付款支付时间差mean'].value_counts()
# d41.loc[d4['分期付款实际支付时间']>0, '分期付款实际支付时间'].value_counts()

# '是否逾期'
d41['是否逾期'] = d41['分期付款支付时间差mean'].apply(lambda x: 1 if x > 0 else 0 )
d41['是否逾期'].value_counts()

0    5444576
1     274643
Name: 是否逾期, dtype: int64

In [93]:
### d4 group by phase 2 ###

d42_agg_dict = {
    '本期还款金额mean':['sum'],
    '本期贷款金额sum':['sum'],
    '分期付款支付时间差mean':['mean'],
    '是否逾期':['mean','sum'] # mean-逾期占比；sum-累计逾期
}
d42 = d41.groupby(['申请编号','历史申请编号']).agg(d42_agg_dict)
d42.columns = pd.Series(d42.columns.tolist()).apply(pd.Series).sum(axis=1)
d42.shape
d42.head()


(445770, 5)

本期还款金额meansum  本期贷款金额sumsum  分期付款支付时间差meanmean  是否逾期mean  是否逾期sum
申请编号 历史申请编号                                                                   
0    779169        11538.0       16472.0         -11.000000  0.083333        1
1    313225         7428.5        8376.0          -6.250000  0.000000        0
     753009        11712.0       11712.0          -3.083333  0.000000        0
2    702654        57934.0       65493.0          -2.458333  0.000000        0
     710924        38610.0       38610.0          -3.466667  0.066667        1

In [94]:
### d4 自定义特征

d42.eval('本期还款金额ms比贷款金额ss = 本期还款金额meansum / 本期贷款金额sumsum', inplace=True)
d42.columns.to_list()

['本期还款金额meansum',
 '本期贷款金额sumsum',
 '分期付款支付时间差meanmean',
 '是否逾期mean',
 '是否逾期sum',
 '本期还款金额ms比贷款金额ss']

In [95]:
### d4 group by phase 3 ###

d43 = d42.groupby('申请编号').agg({
    '本期还款金额ms比贷款金额ss':['mean'],
    '本期还款金额meansum':['sum'],
    '本期贷款金额sumsum':['sum'],
    '是否逾期sum':['sum','mean'], # 是否逾期sumsum:历史累计逾期数; 是否逾期summean:历史累计逾期占比
    '是否逾期mean':['mean'], # 逾期meanmean:逾期占比均值
})
d43.columns = pd.Series(d43.columns.tolist()).apply(pd.Series).sum(axis=1)
'''d43'''
d43.shape
d43.head()


'd43'

(153309, 6)

,本期还款金额ms比贷款金额ssmean,本期还款金额meansumsum,本期贷款金额sumsumsum,是否逾期sumsum,是否逾期summean,是否逾期meanmean
申请编号,,,,,,
0,0.700461,11538.0,16472.0,1,1.00,0.083333
1,0.943440,19140.5,20088.0,0,0.00,0.000000
2,0.971146,221145.0,228704.0,1,0.25,0.016667
3,1.000000,58938.0,58938.0,0,0.00,0.000000
5,0.925340,51451.5,52856.0,0,0.00,0.000000


In [96]:
### d4 group by phase 4 ###

d4_agg_dict = {}
d4_cols = {}
# 金额型
d4_cols['curr'] = [
    '本期贷款金额',
    '本期还款金额']


# 时长型
d4_cols['time_span'] = [
    '分期付款应支付时间',
    '分期付款实际支付时间']

# 计数型
d4_cols['cnt'] = [
    '有支付行为的分期付款']

# 数值型 - 金额
curr_agg = ['mean', 'max', 'min']
for col in d4_cols['curr']:
    d4_agg_dict[col] = curr_agg

# 数值型 - 时长 
time_span_agg = ['mean', 'max', 'min']
for col in d4_cols['time_span']:
    d4_agg_dict[col] = time_span_agg

d4 = d4.groupby('申请编号').agg(d4_agg_dict)
d4.columns = pd.Series(d4.columns.tolist()).apply(pd.Series).sum(axis=1)
'''d4'''
d4.shape

d4 = pd.merge(d4, d43, how='inner', on='申请编号')
'''d4'''
d4.shape
d4.head()

'd4'

(153309, 12)

'd4'

(153309, 18)

,本期贷款金额mean,本期贷款金额max,本期贷款金额min,本期还款金额mean,本期还款金额max,本期还款金额min,分期付款应支付时间mean,分期付款应支付时间max,分期付款应支付时间min,分期付款实际支付时间mean,分期付款实际支付时间max,分期付款实际支付时间min,本期还款金额ms比贷款金额ssmean,本期还款金额meansumsum,本期贷款金额sumsumsum,是否逾期sumsum,是否逾期summean,是否逾期meanmean
申请编号,,,,,,,,,,,,,,,,,,
0,1098.133333,1100.0,1072.0,880.266667,1100.0,12.0,-1600.000000,-1480,-1744,-1610.333333,-1490.0,-1758.0,0.700461,11538.0,16472.0,1,1.00,0.083333
1,803.520000,976.0,635.0,778.520000,976.0,242.0,-267.920000,-19,-518,-272.480000,-23.0,-527.0,0.943440,19140.5,20088.0,0,0.00,0.000000
2,5578.146341,39611.0,2574.0,5455.414634,39611.0,607.0,-331.829268,-4,-641,-334.439024,-28.0,-648.0,0.971146,221145.0,228704.0,1,0.25,0.016667
3,4911.500000,4912.0,4906.0,4911.500000,4912.0,4906.0,-252.000000,-120,-384,-267.000000,-134.0,-396.0,1.000000,58938.0,58938.0,0,0.00,0.000000
5,1957.629630,17350.0,33.0,1923.222222,17350.0,33.0,-1107.925926,-222,-2306,-1111.407407,-222.0,-2306.0,0.925340,51451.5,52856.0,0,0.00,0.000000


In [97]:
### Merge d12 and d4 ###
'''d12'''
d12.shape
'''d4'''
d4.shape

d12_d4 = pd.merge(d12['申请编号'], d4, how='left', on='申请编号')

'''d12_d4'''
d12_d4.shape
# print_na(d12_d4)

#TODO 空值插补0
d12_d4 = d12_d4.fillna(0)

'd12'

(161511, 62)

'd4'

(153309, 18)

'd12_d4'

(161511, 19)

In [98]:
### Merge d12_d3a_all and d12_d4 ###

d12_d3a_all.shape
d12_d4.shape
d1234 = pd.merge(d12_d3a_all, d12_d4, how='left', on='申请编号')
d1234.shape
print_na(d1234)

(161511, 391)

(161511, 19)

(161511, 409)

In [35]:
prefix = 'train'
data_train = pd.merge(d01['申请编号'], d1234, on='申请编号', how='left')
data_train.shape
data_train.to_csv(f'./tmp/{prefix}_d1234.csv')

prefix = 'A'
data_test = pd.merge(d05['申请编号'], d1234, on='申请编号', how='left')
data_test.shape
data_test.to_csv(f'./tmp/{prefix}_d1234.csv')

(140000, 405)

(21511, 405)

In [36]:
'''All Done'''
data_train.columns.to_list()

申请编号
贷款类型
信用额度
贷款年金
商品价格
陪同申请人
出生日期距申请日期天数
工作日期距申请日期天数
注册日期距申请日期天数
身份认证日期距申请日期天数
是否提供手机号
是否提供电话
手机号是否有效
是否提供email
申请周内日
申请时点
外部评分
申请人是否额外提供了文件2
申请人是否额外提供了文件3
申请人是否额外提供了文件4
申请人是否额外提供了文件5
申请人是否额外提供了文件6
申请人是否额外提供了文件7
申请人是否额外提供了文件8
申请人是否额外提供了文件9
贷款申请前1小时内征信查询次数
贷款申请前1天内征信查询次数
贷款申请前1周内征信查询次数
贷款申请前1个月内征信查询次数
贷款申请前1个季度内征信查询次数
贷款申请前1年内征信查询次数
性别
是否有车
是否有房
孩子个数
客户收入
收入类型
教育程度
婚姻状态
居住状态
居住地人口密集度
有车时间
职业
家庭成员数
居住信息1
居住信息2
居住信息3
居住信息4
客户居住地评分1
客户居住地评分2
地址是否一致标志1
地址是否一致标志2
地址是否一致标志3
地址是否一致标志4
地址是否一致标志5
地址是否一致标志6
单位类型
社交圈违约信息2_2
社交圈违约信息2_1
社交圈违约信息1_2
社交圈违约信息1_1
最近一次换手机号码距申请日天数
历史申请编号count_x
贷款年金mean
贷款年金max
贷款年金min
申请额度mean
申请额度max
申请额度min
信用额度mean
信用额度max
信用额度min
首付金额mean
首付金额max
首付金额min
商品价格mean
商品价格max
商品价格min
拟实际应还贷款金额mean
拟实际应还贷款金额max
拟实际应还贷款金额min
拟实际应还贷款金额sum
信用额度比mean
信用额度比max
信用额度比min
历史贷款授信距本次申请时间mean
历史贷款授信距本次申请时间max
历史贷款授信距本次申请时间min
首付时间mean
首付时间max
首付时间min
首付率mean
首付率max
首付率min
利率1mean
利率1max
利率1min
利率2mean
利率2max
利率2min
贷款类型_0sum
贷款类型_0mean
贷款类型_1sum
贷款类型_1mean
贷款类型_Asum
贷款类型_Amean
贷款用途_